In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR="/data/lastfm-dataset-360K/"

In [2]:
!ls {DATA_DIR}

README.txt	 usersha1-artmbid-artname-plays.tsv
mbox_sha1sum.py  usersha1-profile.tsv


In [3]:
!head {DATA_DIR}/usersha1-artmbid-artname-plays.tsv

00000c289a1829a808ac09c00daf10bc3c4e223b	3bd73256-3905-4f3a-97e2-8b341527f805	betty blowtorch	2137
00000c289a1829a808ac09c00daf10bc3c4e223b	f2fb0ff0-5679-42ec-a55c-15109ce6e320	die ��rzte	1099
00000c289a1829a808ac09c00daf10bc3c4e223b	b3ae82c2-e60b-4551-a76d-6620f1b456aa	melissa etheridge	897
00000c289a1829a808ac09c00daf10bc3c4e223b	3d6bbeb7-f90e-4d10-b440-e153c0d10b53	elvenking	717
00000c289a1829a808ac09c00daf10bc3c4e223b	bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8	juliette & the licks	706
00000c289a1829a808ac09c00daf10bc3c4e223b	8bfac288-ccc5-448d-9573-c33ea2aa5c30	red hot chili peppers	691
00000c289a1829a808ac09c00daf10bc3c4e223b	6531c8b1-76ea-4141-b270-eb1ac5b41375	magica	545
00000c289a1829a808ac09c00daf10bc3c4e223b	21f3573f-10cf-44b3-aeaa-26cccd8448b5	the black dahlia murder	507
00000c289a1829a808ac09c00daf10bc3c4e223b	c5db90c4-580d-4f33-b364-fbaa5a3a58b5	the murmurs	424
00000c289a1829a808ac09c00daf10bc3c4e223b	0639533a-0402-40ba-b6e0-18b067198b73	lunachicks	403


In [4]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .getOrCreate()
)

In [5]:
import os
import pyspark.sql.functions as sql_func

plays = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "usersha1-artmbid-artname-plays.tsv"),
        header=False,
        inferSchema=True,
        sep='\t'
    )
    .toDF("user", "artist", "artist_name", "plays")
    .where("LENGTH(artist) >= 35")
    .where("LENGTH(user) == 40")
    .where("plays IS NOT NULL")
    .drop("artist_name")
)

In [6]:
# построим индексы пользователей и исполнителей
user_index = (
    plays
    .select("user")
    .distinct()
    .coalesce(1)
    .select(
        "user",
        sql_func.monotonically_increasing_id().alias("user_id")
    )
)
artist_index = (
    plays
    .select("artist")
    .distinct()
    .coalesce(1)
    .select(
        "artist",
        sql_func.monotonically_increasing_id().alias("artist_id")
    )
)

In [7]:
# перекодируем наш набор данных с использованием
# числовых индексов вместо строковых
triples = (
    plays
    .join(user_index, "user")
    .join(artist_index, "artist")
    .select(
        "user_id",
        "artist_id",
        sql_func.log(
            1 + sql_func.col("plays")
        ).alias("plays")
    )
    .cache()
)
triples.show(n=5)

+-------+---------+-----------------+
|user_id|artist_id|            plays|
+-------+---------+-----------------+
|   3932|      221|5.303304908059076|
|   9048|      221|4.543294782270004|
|  15430|      221|5.049856007249537|
|  16664|      221|6.037870919922137|
|  22832|      221|4.990432586778736|
+-------+---------+-----------------+
only showing top 5 rows



In [8]:
# сохраним тройки на диск для дальнейшего использования
(
    triples
    .write
    .mode("overwrite")
    .parquet("/data/other/user_item_lastfm.parquet")
)

In [9]:
# соберём индексы в виде Python-объектов
i = triples.select("user_id").rdd.map(lambda row: row.user_id).collect()
j = triples.select("artist_id").rdd.map(lambda row: row.artist_id).collect()
data = triples.select("plays").rdd.map(lambda row: row.plays).collect()
# больше Spark нам не понадобится, так что освободим память
spark.stop()

In [10]:
print("номера строк (индексы пользователей):", i[:5])
print("номера столбцов (индексы исполнителей):", j[:5])
print("элементы матрицы (количества прослушиваний):", data[:5])

номера строк (индексы пользователей): [3932, 9048, 15430, 16664, 22832]
номера столбцов (индексы исполнителей): [221, 221, 221, 221, 221]
элементы матрицы (количества прослушиваний): [5.303304908059076, 4.543294782270004, 5.049856007249537, 6.037870919922137, 4.990432586778736]


In [11]:
from scipy.sparse import coo_matrix
import numpy as np
import gc

# соберём разреженную матрицу из полученных данных
user_item_matrix = coo_matrix((data, (i, j)), dtype=np.float16)
# от самих данных избавимся, чтобы не занимать лишнюю память
del i, j, data
gc.collect()

279

In [12]:
# разреженные матрицы - довольно удобный объект
print("размерности матрицы:", user_item_matrix.shape)
print("количество ненулевых элементов:", user_item_matrix.nnz)
print("доля ненулевых элементов:",
      user_item_matrix.nnz / user_item_matrix.shape[0] / user_item_matrix.shape[1])

размерности матрицы: (359337, 160163)
количество ненулевых элементов: 17332977
доля ненулевых элементов: 0.00030116814091700147
